In [5]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import datetime
import time
import datetime as dt
import pytz

import logging

from scipy.signal import savgol_filter, argrelextrema, find_peaks

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Functions for getting price data

In [6]:
def granular(grain='M1'):
    
    timeframes_mapping = {
    'M1': [mt5.TIMEFRAME_M1, 1],
    'M2': [mt5.TIMEFRAME_M2, 2],
    'M3': [mt5.TIMEFRAME_M3, 3],
    'M4': [mt5.TIMEFRAME_M4, 4],
    'M5': [mt5.TIMEFRAME_M5, 5],
    'M6': [mt5.TIMEFRAME_M6, 6],
    'M10': [mt5.TIMEFRAME_M10, 10],
    'M12': [mt5.TIMEFRAME_M12, 12],
    'M15': [mt5.TIMEFRAME_M15, 15],
    'M30': [mt5.TIMEFRAME_M30, 30],
    'H1': [mt5.TIMEFRAME_H1, 60],
    'H2': [mt5.TIMEFRAME_H2, 120],
    'H3': [mt5.TIMEFRAME_H3, 180],
    'H4': [mt5.TIMEFRAME_H4, 240],
    'H6': [mt5.TIMEFRAME_H6, 360],
    'H8': [mt5.TIMEFRAME_H8, 480],
    'H12': [mt5.TIMEFRAME_H12, 720],
    'D1': [mt5.TIMEFRAME_D1, 1440]    
    }

    return timeframes_mapping[grain][0]

def fetch_candles(pair_name,  start_date, end_date, granularity, price):       
    grain = granular(granularity)
    
    if price == 'Price':
        price = mt5.copy_rates_range(pair_name, grain, start_date, end_date)
        price = pd.DataFrame(price)
        price['time'] = pd.to_datetime(price['time'], unit='s')
        price = price.rename(columns={'time': 'date', 'tick_volume': 'volume'})
        #price = price.set_index('date')

    elif price == 'Tick':
        price = mt5.copy_ticks_range(pair_name, start_date, end_date, mt5.COPY_TICKS_ALL)
        price = pd.DataFrame(price)
        price['date'] = pd.to_datetime(price['time_msc'], unit='ms')
        price.set_index('date', inplace=True)
        price.drop('time', axis=1, inplace=True)
        price.drop('time_msc', axis=1, inplace=True)
        price = price[price.columns[price.any()]]
        #price = price.set_index('date')   
    return price

# Functions Concerned with getting Price data
def collect_price(pair, days=5, granularity='M1',  timezone='Etc/GMT-2', price='Price'):    
    zone = pytz.timezone(timezone)
    now = dt.datetime.now(tz=zone)
    dayz = days
    start_date = now - dt.timedelta(days=dayz)

    if start_date.weekday() == 6:
        start_date -=dt.timedelta(days=2)

    elif start_date.weekday() == 5:
        start_date -=dt.timedelta(days=1)

    date_from = start_date.replace(hour=0, minute=0, second=0, microsecond=0)
    date_to = dt.datetime(now.year, now.month, now.day, now.hour, now.minute, now.second)
    
    # collect candles
    # ap = data_api.DataApi()
    price = fetch_candles(pair,granularity=granularity,start_date=date_from,end_date=date_to,price=price)
    
    if len(price) > 0:
        final_df = price
    elif len(price) == 0:
        print('ERROR', pair, granularity, date_from, date_to)
        
    # print(f'{pair} {granularity} {final_df.iloc[0].date} {final_df.iloc[-1].date}' )
    return final_df

# Function to get the latest price
def get_price(symbol):
    tick = mt5.symbol_info_tick(symbol)
    return tick.bid, tick.ask

# Functions for getting market symbols and instruments
def get_instruments_df():
    instrument_data = []
    ins = mt5.symbols_get()
    for item in range(len(ins)):
        new_ob = dict(
            name = ins[item].name,
            type = ins[item].path,
            displayName = ins[item].description,
            pipLocation = ins[item].digits,
            marginRate = ins[item].volume_min
        )
        instrument_data.append(new_ob)
    instrument_df = pd.DataFrame.from_dict(instrument_data)
    return instrument_df

def market_types(Synth_list):    
    lis = []
    for i in range(len(Synth_list)):
        j = 0

        lend = ""
        while Synth_list[i].path[j] != "\\":
            
            lend = f'{lend}'+Synth_list[i].path[j]
            j += 1
        lis.append(lend)

    condensed = list(set(lis))

    secondary = []
    for j in range(len(condensed)):
        initial = []
        for i in range(len(lis)):
            if lis[i] == condensed[j]:
                initial.append(Synth_list[i].name)
        secondary.append(initial)
    
    return condensed, secondary

In [7]:
def clean_tick_data(df: pd.DataFrame,
                    n_digits: int,
                    timezone: str = 'Etc/GMT-2'
                    ):
    """
    Clean and validate Forex tick data with comprehensive quality checks.

    Args:
        df: DataFrame containing tick data with bid/ask prices and timestamp index
        n_digits: Number of decimal places in instrument price.
        timezone: Timezone to localize/convert timestamps to (default: 'Etc/GMT-2')

    Returns:
        Cleaned DataFrame or None if empty after cleaning
    """
    if df.empty:
        return None

    df = df.copy(deep=False)  # Work on a copy to avoid modifying the original DataFrame 
    n_initial = df.shape[0] # Store initial row count for reporting

    # 1. Ensure proper datetime index
    # Use errors='coerce' to turn unparseable dates into NaT and then drop them.
    if not isinstance(df.index, pd.DatetimeIndex):
        original_index_name = df.index.name
        df.index = pd.to_datetime(df.index, errors='coerce')
        nan_idx_count = df.index.isnull().sum()
        if nan_idx_count > 0:
            logging.info(f"Dropped {nan_idx_count:,} rows with unparseable timestamps.")
            df = df[~df.index.isnull()]
        if original_index_name:
            df.index.name = original_index_name
    
    if df.empty: # Check if empty after index cleaning
        logging.warning("Warning: DataFrame empty after initial index cleaning")
        return None

    # 2. Timezone handling
    if df.index.tz is None:
        df = df.tz_localize(timezone)
    elif str(df.index.tz) != timezone.upper():
        df = df.tz_convert(timezone)
    
    # 3. Price validity checks
    # Apply rounding and then filtering
    df['bid'] = df['bid'].round(n_digits)
    df['ask'] = df['ask'].round(n_digits)

    # Validate prices
    price_filter = (
        (df['bid'] > 0) &
        (df['ask'] > 0) &
        (df['ask'] > df['bid'])
    )
    
    n_before_price_filter = df.shape[0]
    df = df[price_filter]
    n_filtered_prices = n_before_price_filter - df.shape[0]
    if n_filtered_prices > 0:
        logging.info(f"Filtered {n_filtered_prices:,} ({n_filtered_prices / n_before_price_filter:.2%}) invalid prices.")

    if df.empty: # Check if empty after price cleaning
        logging.warning("Warning: DataFrame empty after price cleaning")
        return None
    
    # Dropping NA values
    initial_rows_before_na = df.shape[0]
    if df.isna().any().any(): # Use .any().any() to check if any NA exists in the whole DF
        na_counts = df.isna().sum()
        na_cols = na_counts[na_counts > 0]
        if not na_cols.empty:
            logging.info(f'Dropped NA values from columns: \n{na_cols}')
            df.dropna(inplace=True)

    n_dropped_na = initial_rows_before_na - df.shape[0]
    if n_dropped_na > 0:
        logging.info(f"Dropped {n_dropped_na:,} ({n_dropped_na / n_before_price_filter:.2%}) rows due to NA values.")

    if df.empty: # Check if empty after NA cleaning
        logging.warning("Warning: DataFrame empty after NA cleaning")
        return None
    
    # 4. Microsecond handling
    if not df.index.microsecond.any():
        logging.warning("Warning: No timestamps with microsecond precision found")
    
    # 5. Duplicate handling
    duplicate_mask = df.index.duplicated(keep='last')
    dup_count = duplicate_mask.sum()
    if dup_count > 0:
        logging.info(f"Removed {dup_count:,} ({dup_count / n_before_price_filter:.2%}) duplicate timestamps.")
        df = df[~duplicate_mask]

    if df.empty: # Check if empty after duplicate cleaning
        logging.warning("Warning: DataFrame empty after duplicate cleaning")
        return None

    # 6. Chronological order
    if not df.index.is_monotonic_increasing:
        logging.info("Sorting DataFrame by index to ensure chronological order.")
        df.sort_index(inplace=True)

    # 7. Final validation and reporting
    if df.empty:
        logging.warning("Warning: DataFrame empty after all cleaning steps.")
        return None
    
    n_final = df.shape[0]
    n_cleaned = n_initial - n_final
    percentage_cleaned = (n_cleaned / n_initial) if n_initial > 0 else 0
    logging.info(f"Cleaned {n_cleaned:,} of {n_initial:,} ({percentage_cleaned:.2%}) datapoints.")

    return df

def set_resampling_freq(timeframe: str) -> str:
    """
    Converts an MT5 timeframe to a pandas resampling frequency.

    Args:
        timeframe (str): MT5 timeframe (e.g., 'M1', 'M5', 'M15', 'H1', 'H4', 'D1', 'W1').

    Returns:
        str: Pandas frequency string.
    """
    timeframe = timeframe.upper()
    nums = [x for x in timeframe if x.isnumeric()]
    if not nums:
        raise ValueError("Timeframe must include numeric values (e.g., 'M1').")
    
    x = int(''.join(nums))
    if timeframe == 'W1':
        freq = 'W-FRI'
    elif timeframe == 'D1':
        freq = 'B'
    elif timeframe.startswith('H'):
        freq = f'{x}H'
    elif timeframe.startswith('M'):
        freq = f'{x}min'
    elif timeframe.startswith('S'):
        freq = f'{x}S'
    else:
        raise ValueError("Valid timeframes include W1, D1, Hx, Mx, Sx.")
    
    return freq

def calculate_ticks_per_period(df: pd.DataFrame, timeframe: str = "M1", method: str = 'median', verbose: bool = True) -> int:
    """
    Dynamically calculates the average number of ticks per given timeframe.

    Args:
        df (pd.DataFrame): Tick data.
        timeframe (str): MT5 timeframe.
        method (str): 'median' or 'mean' for the calculation.
        verbose (bool): Whether to print the result.

    Returns:
        int: Rounded average ticks per period.
    """
    freq = set_resampling_freq(timeframe)
    resampled = df.resample(freq).size()
    fn = getattr(np, method)
    num_ticks = fn(resampled.values)
    num_rounded = int(np.round(num_ticks))
    num_digits = len(str(num_rounded)) - 1
    rounded_ticks = int(round(num_rounded, -num_digits))
    rounded_ticks = max(1, rounded_ticks)
    
    if verbose:
        t0 = df.index[0].date()
        t1 = df.index[-1].date()
        logging.info(f"From {t0} to {t1}, {method} ticks per {timeframe}: {num_ticks:,} rounded to {rounded_ticks:,}")
    
    return rounded_ticks

def flatten_column_names(df):
    '''
    Joins tuples created by dataframe aggregation 
    with a list of functions into a unified name.
    '''
    return ["_".join(col).strip() for col in df.columns.values]

def make_bar_type_grouper(
        df: pd.DataFrame,
        bar_type: str = 'tick',
        bar_size: int = 100,
        timeframe: str = 'M1'
) -> tuple[pd.core.groupby.generic.DataFrameGroupBy, int]:
    """
    Create a grouped object for aggregating tick data into time/tick/dollar/volume bars.

    Args:
        df: DataFrame with tick data (index should be datetime for time bars).
        bar_type: Type of bar ('time', 'tick', 'dollar', 'volume').
        bar_size: Number of ticks/dollars/volume per bar (ignored for time bars).
        timeframe: Timeframe for resampling (e.g., 'H1', 'D1', 'W1').

    Returns:
        - GroupBy object for aggregation
        - Calculated bar_size (for tick/dollar/volume bars)
    """
    # Create working copy (shallow is sufficient)
    df = df.copy(deep=False)  # OPTIMIZATION: Shallow copy here only once
    
    # Ensure DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        try:
            df = df.set_index('time')
        except KeyError:
            raise TypeError("Could not set 'time' as index")

    # Sort if needed
    if not df.index.is_monotonic_increasing:
        df = df.sort_index()

    # Time bars
    if bar_type == 'time':
        freq = set_resampling_freq(timeframe)
        bar_group = (df.resample(freq, closed='left', label='right') # includes data upto, but not including, the end of the period
                    if not freq.startswith(('B', 'W')) 
                    else df.resample(freq))
        return bar_group, 0  # bar_size not used

    # Dynamic bar sizing
    if bar_size == 0:
        if bar_type == 'tick':
            bar_size = calculate_ticks_per_period(df, timeframe)
        else:
            raise NotImplementedError(f"{bar_type} bars require non-zero bar_size")

    # Non-time bars
    df['time'] = df.index  # Add without copying
    
    if bar_type == 'tick':
        bar_id = np.arange(len(df)) // bar_size
    elif bar_type in ('volume', 'dollar'):
        if 'volume' not in df.columns:
            raise KeyError(f"'volume' column required for {bar_type} bars")
        
        # Optimized cumulative sum
        cum_metric = (df['volume'] * df['bid'] if bar_type == 'dollar' 
                      else df['volume'])
        cumsum = cum_metric.cumsum()
        bar_id = (cumsum // bar_size).astype(int)
    else:
        raise NotImplementedError(f"{bar_type} bars not implemented")

    return df.groupby(bar_id), bar_size

def make_bars(tick_df: pd.DataFrame,
              bar_type: str = 'tick',
              bar_size: int = 0,
              timeframe: str = 'M1',
              price: str = 'midprice',
              verbose=True):
    '''
    Create OHLC data by sampling ticks using timeframe or a threshold.

    Parameters
    ----------
    tick_df: pd.DataFrame
        tick data
    bar_type: str
        type of bars to create from ['tick', 'time', 'volume', 'dollar']
    bar_size: int 
        default 0. bar_size when bar_type != 'time'
    timeframe: str
        MT5 timeframe (e.g., 'M5', 'H1', 'D1', 'W1').
        Used for time bars, or for tick bars if bar_size = 0.
    price: str
        default midprice. If 'bid_ask', columns (bid_open, ..., bid_close), 
        (ask_open, ..., ask_close) are included.
    verbose: bool
        print information about the data

    Returns
    -------
    pd.DataFrame with columns [open, high, low, close, median_price, tick_volume, volume]
    '''    
    if 'midprice' not in tick_df:
        tick_df['midprice'] = (tick_df['bid'] + tick_df['ask']) / 2

    bar_group, bar_size_ = make_bar_type_grouper(tick_df, bar_type, bar_size, timeframe)
    ohlc_df = bar_group['midprice'].ohlc().astype('float64')
    ohlc_df['tick_volume'] = bar_group['bid'].count() if bar_type != 'tick' else bar_size_
    
    if price == 'bid_ask':
        # Aggregate OHLC data for every bar_size rows
        bid_ask_df = bar_group.agg({k: 'ohlc' for k in ('bid', 'ask')})
        # Flatten MultiIndex columns
        col_names = flatten_column_names(bid_ask_df)
        bid_ask_df.columns = col_names
        ohlc_df = ohlc_df.join(bid_ask_df)

    if 'volume' in tick_df:
        ohlc_df['volume'] = bar_group['volume'].sum()

    if bar_type == 'time':
        ohlc_df.ffill(inplace=True)
    else:
        end_time =  bar_group['time'].last()
        ohlc_df.index = end_time + pd.Timedelta(microseconds=1) # ensure end time is after event
	    # ohlc_df.drop('time', axis=1, inplace=True) # Remove 'time' column


        # drop last bar due to insufficient ticks
        if len(tick_df) % bar_size_ > 0: 
            ohlc_df = ohlc_df.iloc[:-1]

    if verbose:
        if bar_type != 'time':
            tm = f'{bar_size_:,}'
            if bar_type == 'tick' and bar_size == 0:
                tm = f'{timeframe} - {bar_size_:,} ticks'
            timeframe = tm
        print(f'\nTick data - {tick_df.shape[0]:,} rows')
        print(f'{bar_type}_bar {timeframe}')
        ohlc_df.info()
    
    # Remove timezone info from DatetimeIndex
    try:
	    ohlc_df = ohlc_df.tz_convert(None)
     
    except:
	    pass
    
    return ohlc_df

## Functions for Trade Management and entering trades

In [8]:
# Function to place a trade
def place_trade(symbol, lot, direction, stop_loss_pips, take_profit_pips):
    price = get_price(symbol)
    deviation = 10
    trade_type = mt5.ORDER_TYPE_BUY if direction == "buy" else mt5.ORDER_TYPE_SELL
    # sl = price[0] - stop_loss_pips * 0.0001 if direction == "buy" else price[0] + stop_loss_pips * 0.0001
    # tp = price[0] + take_profit_pips * 0.0001 if direction == "buy" else price[0] - take_profit_pips * 0.0001
    
    sl = stop_loss_pips if direction == "buy" else stop_loss_pips
    tp = take_profit_pips if direction == "buy" else take_profit_pips
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": trade_type,
        "price": price[1] if direction == "buy" else price[0],
        "sl": sl,
        "tp": tp,
        "deviation": deviation,
        "magic": 100001,
        "comment": "Martingale trade",
        "type_time": mt5.ORDER_TIME_GTC
    }
    return mt5.order_send(request)

# Function to check last trade outcome
def check_last_trade():
    history = mt5.history_deals_get()
    if history:
        last_trade = list(history)[-1]
        return last_trade.profit < 0  # Return True if last trade was a loss
    return False

# Function to check daily profit
def get_daily_profit():
    today = datetime.datetime.now().date()
    deals = mt5.history_deals_get(datetime.datetime(today.year, today.month, today.day), datetime.datetime.now())
    return sum(deal.profit for deal in deals if deal.profit is not None)

In [9]:
import indicators as indie
import Set_functions as SF

In [10]:
def get_candles_frame(Symbol, entry_strat='MA_cross', short_days=6, long_days=6, short_tf='M1', mid_tf='M15', long_tf='M30', high_tf='M30', X_tf='H1'):

    # stop_level = SF.stop_levels(Symbol, Synth_list)[0]
    # lot_size_min = SF.stop_levels(Symbol, Synth_list)[1]
    # lot_size_max = SF.stop_levels(Symbol, Synth_list)[2]

    df = SF.get_tick(pair=Symbol, days=short_days, granularity=short_tf)
    # df_01 = df.copy()
    df_02 = SF.get_tick(pair=Symbol, days=short_days, granularity=mid_tf)
    df_03 = SF.get_tick(pair=Symbol, days=long_days, granularity=long_tf)
    df_04 = SF.get_tick(pair=Symbol, days=long_days, granularity=high_tf)
    df_05 = SF.get_tick(pair=Symbol, days=10, granularity=X_tf)
    
    df_01a = SF.get_tick(pair=Symbol, days=short_days, granularity="M5")
    
    df['ATR'] = indie.atr(df, 7)

    data = df.rename(columns={'close': 'Close', 'low': 'Low', 'high': 'High', 'date': 'Date', 'open' : 'Open'})
    
        # Assume df_m15 has datetime index and OHLC data
    df_m5 = indie.compute_cci(df_02.rename(columns={'close': 'Close', 'low': 'Low', 'high': 'High', 'date': 'Date', 'open' : 'Open'}), n=40)
    # df_m5 = df_m5.dropna(subset=['cci_40'])

    df_m15 = indie.compute_cci(df_03.rename(columns={'close': 'Close', 'low': 'Low', 'high': 'High', 'date': 'Date', 'open' : 'Open'}), n=40)
    # df_m15 = df_m15.dropna(subset=['cci_40'])

    df_h1 = indie.compute_cci(df_05.rename(columns={'close': 'Close', 'low': 'Low', 'high': 'High', 'date': 'Date', 'open' : 'Open'}), n=40)
    # df_h1 = df_h1.dropna(subset=['cci_40'])

    df['h1_cross'] = pd.DataFrame(SF.track_cci_cross_moves_with_time(df_h1)).set_index(0).reindex(df.index, method='ffill')
    df['m15_cross'] = pd.DataFrame(SF.track_cci_cross_moves_with_time(df_m15)).set_index(0).reindex(df.index, method='ffill')
    df['m5_cross'] = pd.DataFrame(SF.track_cci_cross_moves_with_time(df_m5)).set_index(0).reindex(df.index, method='ffill')
    
    # Create a preliminary signal for Stochastic Entries
    if entry_strat == "Simple":
        df["Signal"] = SF.create_signals(data, algo=SF.simple_signal)

    elif entry_strat == "ICT":   
        df["Signal"] = SF.create_signals(df, algo=SF.ICT_signal)

    elif entry_strat == "Mharris":   
        df["Signal"] = SF.create_signals(df, algo=SF.Mharris_signal)

    elif entry_strat == "Stoch":
        full_stoch = indie.stoch(df, 50)
        df['D'] = full_stoch.slow_d
        df['K'] = full_stoch.slow_k
        df["Signal"] = SF.create_signals(df, algo=SF.stoch_signal)
        
    elif entry_strat == "MA_cross":
        MA1 = indie.rolling_mean(df.close, 2)
        MA5 = indie.rolling_mean(df.close, 6)
        df['crause'] = SF.crossover_lines(df, MA1, MA5)['positions']
        df["Signal"] = SF.create_signals(df, algo=SF.cross_signal)
        
    elif entry_strat == "Peaks_Valleys":
        df_hl = df_02.rename(columns={'close': 'Close', 'low': 'Low', 'high': 'High', 'date': 'Date', 'open' : 'Open'})
        df_hl = df_hl.set_index('Date')
        highs_lows = SF.peak_vall(df_hl)
        df["Highs_lows"] = highs_lows['Signal'].reindex(df.index)
        df["Signal"] = SF.create_signals(df, algo=SF.p_v_signal)

    # df = df.set_index('date')

    # Creating Z Wave with it's cyclic directional arrows
    line_04, line_0_dated4, line_14, line_1_dated4 = SF.smooth_frame_lines(df_04, line='MACD')
    k_wave = line_1_dated4.line0
    trend_d_ind4, trend_d_loc_main4, trend_u_ind4, trend_u_loc_main4 = SF.waveler_01(k_wave)
    
    circle1c = pd.DataFrame()
    circle2c = pd.DataFrame()

    circle1c['values'] = trend_d_loc_main4
    circle1c['direction'] = 1
    circle2c['values'] = trend_u_loc_main4
    circle2c['direction'] = 2

    circlec = pd.concat([circle1c, circle2c]).sort_index()
    circc = circlec

    # Creating Z Wave with it's cyclic directional arrows
    line_03, line_0_dated3, line_13, line_1_dated3 = SF.smooth_frame_lines(df_03, line='MACD')
    z_wave = line_1_dated3.line0
    trend_d_ind3, trend_d_loc_main3, trend_u_ind3, trend_u_loc_main3 = SF.waveler_01(z_wave)

    circle1 = pd.DataFrame()
    circle2 = pd.DataFrame()

    circle1['values'] = trend_d_loc_main3
    circle1['direction'] = 1
    circle2['values'] = trend_u_loc_main3
    circle2['direction'] = 2

    circle = pd.concat([circle1, circle2]).sort_index()
    circ = circle

    line_02, line_0_dated2, line_12, line_1_dated2 = SF.smooth_frame_lines(df_02, line='MACD')
    y_wave = line_1_dated2.line0
    trend_d_ind2, trend_d_loc_main2, trend_u_ind2, trend_u_loc_main2 = SF.waveler_01(y_wave)

    circle1a = pd.DataFrame()
    circle2a = pd.DataFrame()

    circle1a['values'] = trend_d_loc_main2
    circle1a['direction'] = 1
    circle2a['values'] = trend_u_loc_main2
    circle2a['direction'] = 2

    circlea = pd.concat([circle1a, circle2a]).sort_index()
    circa = circlea

    # df_03 = df_03.set_index('date')
    peaks, valleys, stretch_filtered, Divergence, Div_date, MA_13, MA_34, norm_stretch_df, Points = SF.divergence_norm(df_03)
    # peaks, valleys, stretch_filtered, Divergence, Div_date, MA_13, MA_34, norm_stretch_df, Points = SF.divergence_norm_realtime(df_03)
    
    # df_02 = df_02.set_index('date')
    cCCi = indie.compute_cci(df_02.rename(columns={'close': 'Close', 'low': 'Low', 'high': 'High', 'date': 'Date', 'open' : 'Open'}), n=25)

    # df['Z'] = cCCi.reindex(df.index, method='ffill')#SF.calculate_z_score(df_01['close'].reindex(df.index, method='ffill'))#
    df['cci_02'] = cCCi.reindex(df.index, method='ffill')
    
    cci_03 = indie.compute_cci(df_04.rename(columns={'close': 'Close', 'low': 'Low', 'high': 'High', 'date': 'Date', 'open' : 'Open'}), n=100)
    
    cci_04 = indie.compute_cci(df_05.rename(columns={'close': 'Close', 'low': 'Low', 'high': 'High', 'date': 'Date', 'open' : 'Open'}), n=100)
    
    df['cci_03'] = cci_03.reindex(df.index).interpolate()
    
    df['cci_M30'] = SF.mark_bias(df_04, cci_col=cci_03, upper=100, lower=-100)['bias'].reindex(df.index, method='ffill')
    
    df['cci_H4'] = SF.mark_bias(df_05, cci_col=cci_04, upper=100, lower=-100)['bias'].reindex(df.index, method='ffill')
    
    # Lines = abs(norm_stretch_df['norm_stretch'])
    # Line_AB = norm_stretch_df['stretch_filtered']
    
    Deviate = SF.dev_indicator(df_03, df_03)
    
    # Points_01 = SF.double_point(Points)
    
    # df['Points_01'] = Points_01['Divergence_01'].ffill().reindex(df.index, method='ffill')
    
    # df_05 = df_05.set_index('date')
    peaks_01, valleys_01, stretch_filtered_01, Divergence_01, Div_date_01, MA_13_01, MA_34_01, norm_stretch_df_01, Points_01 = SF.divergence_norm(df_05)
    
    # df['Sides'] = norm_stretch_df_01['stretch_filtered'].reindex(df.index, method='ffill')
    
    # df['Lines'] = norm_stretch_df_01['stretch_filtered'].ffill().diff().reindex(df.index, method='ffill')
    
    df['Z'] = norm_stretch_df['norm_stretch'].reindex(df.index, method='ffill')#SF.calculate_z_score(df_01['close'].reindex(df.index, method='ffill'))##cCCi#
    df['Confirm'] = norm_stretch_df['stretch_filtered'].ffill().diff().reindex(df.index, method='ffill')
    
    centred = np.mean(norm_stretch_df['norm_stretch'])
    up_c = centred + np.std(norm_stretch_df['norm_stretch'])
    down_c = centred - np.std(norm_stretch_df['norm_stretch'])
    

    # Joining the Circle to the DataFrames
    df['HTF_cyc'] = circ['direction'].reindex(df.index, method='ffill')
    df['MTF_cyc'] = circa['direction'].reindex(df.index, method='ffill')
    
    df['deviate'] = Deviate[0].reindex(df.index, method='ffill')

    # Creating the signals for the trades
    Final_signal = []
    for i in range(len(df)):
        ind = i-1
        # if (df['Points_01'].iloc[ind] == 'Down') & (df['HTF_cyc'].iloc[ind] == 2) :#& (df['MTF_cyc'].iloc[ind] == 2):
        #     Final_signal.append(1)
            
        # elif (df['Points_01'].iloc[ind] == 'Up') & (df['HTF_cyc'].iloc[ind] == 1) :#& (df['MTF_cyc'].iloc[ind] == 1):
        #     Final_signal.append(2)
            
        # else:
        #     Final_signal.append(0)
            
        # if (df['Signal'].iloc[ind-1] == 1) & (df['Sides'].iloc[ind] < centred) & (df['Sides'].iloc[ind] < up_c) & (df['HTF_cyc'].iloc[ind] == 2) & (df['MTF_cyc'].iloc[ind] == 2): #(df['Lines'].iloc[ind] <= 0)
        #     Final_signal.append(1)
            
        # elif (df['Signal'].iloc[ind-1] == 2) & (df['Sides'].iloc[ind] > centred) & (df['Sides'].iloc[ind] > down_c) & (df['HTF_cyc'].iloc[ind] == 1) & (df['MTF_cyc'].iloc[ind] == 1): #(df['Lines'].iloc[ind] >= 0)
        #     Final_signal.append(2)
        
        # if (df['Signal'].iloc[ind-1] == 1) & (df['m15_cross'].iloc[ind] == 'down') & (df['m5_cross'].iloc[ind] == 'down') & (df['MTF_cyc'].iloc[ind] == 2) & (df['deviate'].iloc[ind] == 1): #& (df['HTF_cyc'].iloc[ind] == 1) & (df['h1_cross'].iloc[ind] == 'down')(df['Lines'].iloc[ind] <= 0) (df['Sides'].iloc[ind] < centred) & (df['Sides'].iloc[ind] < up_c)
        #     Final_signal.append(2)
            
        # elif (df['Signal'].iloc[ind-1] == 2)& (df['m15_cross'].iloc[ind] == 'up') & (df['m5_cross'].iloc[ind] == 'up') & (df['MTF_cyc'].iloc[ind] == 1) & (df['deviate'].iloc[ind] == 2): #& (df['HTF_cyc'].iloc[ind] == 2) & (df['h1_cross'].iloc[ind] == 'up') (df['Lines'].iloc[ind] >= 0) (df['Sides'].iloc[ind] > centred) & (df['Sides'].iloc[ind] > down_c)
        #     Final_signal.append(1)
        if (df['Signal'].iloc[ind-1] == 1) & (df['cci_02'].iloc[ind] < -100) & (df['MTF_cyc'].iloc[ind] == 1) & (df['cci_M30'].iloc[ind] == 'up') & (df['cci_H4'].iloc[ind] == 'up') : #  & & ((df['open_zone'].iloc[ind]=='upper') or (df['cci_03'].iloc[ind] >= 100))(df['deviate'].iloc[ind] == 2) & (df['open_range'].iloc[ind] >= open_mean*0.35 + open_mean)& (df['LTF_cyc'].iloc[ind] == 1) & (df['m5_cross'].iloc[ind] == 'up')& (df['m15_cross'].iloc[ind] == 'down')    & (df['Z'].iloc[ind] >=0) & (df['HTF_cyc'].iloc[ind] == 2)& (df['h1_cross'].iloc[ind] == 'down')(df['Lines'].iloc[ind] <= 0) (df['Sides'].iloc[ind] < centred) & (df['Sides'].iloc[ind] < up_c)
            Final_signal.append(2)
            
        elif (df['Signal'].iloc[ind-1] == 2)& (df['cci_02'].iloc[ind] > 100) & (df['MTF_cyc'].iloc[ind] == 2) & (df['cci_M30'].iloc[ind] == 'down')  & (df['cci_H4'].iloc[ind] == 'down'): #& ((df['open_zone'].iloc[ind]=='lower') or (df['cci_03'].iloc[ind] <= -100)) & (df['deviate'].iloc[ind] == 1)  & (df['open_range'].iloc[ind] >= open_mean*0.35 + open_mean) & (df['m5_cross'].iloc[ind] == 'down')& (df['m15_cross'].iloc[ind] == 'up')  & (df['Z'].iloc[ind] <=0) & (df['HTF_cyc'].iloc[ind] == 1) & (df['h1_cross'].iloc[ind] == 'up') (df['Lines'].iloc[ind] >= 0) (df['Sides'].iloc[ind] > centred) & (df['Sides'].iloc[ind] > down_c)
            Final_signal.append(1)    
        
        else:
            Final_signal.append(0)

    df['Final_signal'] = Final_signal
    
    return df, y_wave

## User Logins

In [11]:
# Login details
User = 5228413#20765584#
Pass = "$kyrock3tinG"
Server = "Deriv-Demo"
path_0 = "C:\\Program Files\\MetaTrader 5\\terminal64.exe"

# Initialize connection to MT5
if not mt5.initialize(path=path_0, login=User, server=Server,password=Pass):
    print("Failed to initialize MT5")
    mt5.shutdown()
    
# if not mt5.login(login=User, server=Server,password=Pass):
#     print("login() failed, error code =", mt5.last_error())
#     quit()

## Getting the Markets and Symbols from the broker 

In [12]:
Options_1 =  market_types(mt5.symbols_get())[0]         # Type of markets available with Broker
Options_2 =  market_types(mt5.symbols_get())[1]         # Currenc pair to be selected
        
Options_1

['Stock Indices',
 'Tactical Indices',
 'Skewed Step',
 'Equities',
 'Forex Major',
 'Crash Boom Indices',
 'Step Indices',
 'Trek Indices',
 'Range Break',
 'Energies',
 'Drift Switching Indices',
 'Crypto',
 'Stable Spreads',
 'Volatility Indices',
 'Forex Exotic',
 'Basket Indices',
 'Forex Minor',
 'Multi Step Indices',
 'Conversions',
 'Volatility Switch Indices',
 'ETFs',
 'DEX Indices',
 'Derived Indices',
 'Soft Commodities',
 'Forex Micro',
 'Jump Indices',
 'Metals',
 'Hybrid Indices']

In [13]:
# From the "Options_1" list above, input the Market instruments you want to look at and get a print of the symbols you'll be focusing on
for i in range(len(Options_1)):
    if Options_1[i] == "Forex Major":##"Volatility Indices":'Cent':#'Forex Minor':#'DEX Indices':#"Crypto":#"Metals":#'Equities':#'ETFs':#
        ind = i
        break

# Define a global variable to store the selected index
Market_value = Options_1[ind]
Market_ind = Options_1.index(Market_value)

All_symbols = Options_2[Market_ind]

All_symbols

['AUDJPY',
 'AUDUSD',
 'EURAUD',
 'EURCAD',
 'EURCHF',
 'EURGBP',
 'EURJPY',
 'EURUSD',
 'GBPJPY',
 'GBPUSD',
 'USDCAD',
 'USDCHF',
 'USDJPY',
 'GBPAUD']

In [14]:
# All_symbols.pop(11)

In [15]:
All_symbols

['AUDJPY',
 'AUDUSD',
 'EURAUD',
 'EURCAD',
 'EURCHF',
 'EURGBP',
 'EURJPY',
 'EURUSD',
 'GBPJPY',
 'GBPUSD',
 'USDCAD',
 'USDCHF',
 'USDJPY',
 'GBPAUD']

In [16]:
multiplier_list = np.ones(len(All_symbols))*0.01#[0.0015, 0.005, 0.009, 0.013, 0.02, 0.0015, 0.0035, 0.004, 0.008, 0.01, 0.01, 0.02, 0.013, 0.02, 0.028]
# multiplier_list.pop(11)

In [17]:
multiplier_list

array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01,
       0.01, 0.01, 0.01])

In [18]:
martingale_multiplier = 2.0
sl_multiplier = 100
tp_multiplier = 100
daily_profit_target = 1000.0  # Set your daily profit target



# Martingale trading loop
while True:
    for i in range(len(All_symbols)):
        
        symbol = All_symbols[i]
        
        # Setting up the lot size for each symbol
        stop_level, lot_size_min, lot_size_max = SF.stop_levels(Symbol=symbol, Synth_list=mt5.symbols_get())
        lot_size = 0.05#lot_size_min
        max_lot = lot_size_max
        
        # # Creating a Martingale system
        daily_profit = get_daily_profit()
        # if daily_profit >= daily_profit_target:
        #     print(f"Target reached! Daily Profit: ${daily_profit:.2f}")
        #     break  # Stop trading for the day
        
        if check_last_trade():
            lot_size = min(lot_size * martingale_multiplier, max_lot)
        else:
            lot_size = 0.05#lot_size_min
        
        # Getting the Analysed Price for each symbol
        get, y_wave = get_candles_frame(Symbol=symbol, entry_strat='Simple', short_days=6, long_days=6, short_tf='M1', mid_tf='M15', long_tf='M30', high_tf='M30', X_tf='H1')
        
        centre = np.mean(get['Z'])
        up_c = centre + np.std(get['Z'])
        # down_c = centre - np.std(get['norm_stretch'])

        # SF.detect_zscore_exit_signals(df, level=up_c)
        
        get = SF.detect_zscore_exit_signals(df=get, level=up_c)
        
        peaks, _ = find_peaks(y_wave, distance=15, prominence=0.2)
        valleys, _ = find_peaks(-y_wave, distance=15, prominence=0.2)
        
        # Setting up the stop loss and take profits
        atr = get.ATR.iloc[-1]
        
        if get['Final_signal'].iloc[-1] == 2:
            price = mt5.symbol_info_tick(symbol).ask
            stop_loss_pips = price - (atr * sl_multiplier)
            take_profit_pips = price + (atr * tp_multiplier)
            
            bet = place_trade(symbol, lot_size, "buy", stop_loss_pips, take_profit_pips)
            
            if bet.retcode == 10009:
                print(f"For {symbol}:")
                print(f"Placed a buy trade with {lot_size} lots. Current Daily Profit: ${daily_profit:.2f}") 
            
            elif bet.retcode != 10009:
                print(f"For {symbol}:")
                print(f"Error buy: {bet.retcode}, no trade opened!")          
            
        elif get['Final_signal'].iloc[-1] == 1:
            price = mt5.symbol_info_tick(symbol).bid
            stop_loss_pips = price + (atr * sl_multiplier)
            take_profit_pips = price - (atr * tp_multiplier)
            
            bet = place_trade(symbol, lot_size, "sell", stop_loss_pips, take_profit_pips)  # Example strategy: always buy
            
            if bet.retcode == 10009:
                print(f"For {symbol}:")
                print(f"Placed a sell trade with {lot_size} lots. Current Daily Profit: ${daily_profit:.2f}")
            
            elif bet.retcode != 10009:
                print(f"For {symbol}:")
                print(f"Error sell: {bet.retcode}, no trade opened!")
        
        # Check profit level, so as to close in profit
        positions = mt5.positions_get()
        
        if get['z_exit_signal'].iloc[-1] == 1 or get['z_exit_signal'].iloc[-2] == 1:
            if len(positions) > 0:
                for j in range(len(positions)):
                    if positions[j].symbol == symbol:                   
                        # Order type is a buy
                        if positions[j].type == 0:
                            # SF.close_all_trades(symbol, positions)
                            SF.close_all_trades_01(symbol, positions, type=0)
                            print(f'Symbol {symbol} closed at Profit')
                            
        elif get['z_exit_signal'].iloc[-1] == -1 or get['z_exit_signal'].iloc[-2] == -1:
            if len(positions) > 0:
                for j in range(len(positions)):
                    if positions[j].symbol == symbol:                   
                        # Order type is a sell    
                        if positions[j].type == 1:
                            # SF.close_all_trades(symbol, positions)
                            SF.close_all_trades_01(symbol, positions, type=1)
                            print(f'Symbol {symbol} closed at Profit')
        
        elif (get['cci_M30'].iloc[-1] == 'down') and (get['cci_H4'].iloc[-1] == 'down'):
            if len(positions) > 0:
                for j in range(len(positions)):
                    if positions[j].symbol == symbol:                   
                        # Order type is a buy
                        if positions[j].type == 0:
                            # SF.close_all_trades(symbol, positions)
                            SF.close_all_trades_01(symbol, positions, type=0)
                            print(f'Symbol {symbol} closed at Profit')
                            
        elif (get['cci_M30'].iloc[-1] == 'up') and (get['cci_H4'].iloc[-1] == 'up'):
            if len(positions) > 0:
                for j in range(len(positions)):
                    if positions[j].symbol == symbol:                   
                        # Order type is a sell    
                        if positions[j].type == 1:
                            # SF.close_all_trades(symbol, positions)
                            SF.close_all_trades_01(symbol, positions, type=1)
                            print(f'Symbol {symbol} closed at Profit')
        # print(symbol)                    
        # print(get['cci_M30'].iloc[-1])
        # print(get['cci_H4'].iloc[-1])
        # if len(positions) > 0:
            
        #     for j in range(len(positions)):
                
        #         open_time = pd.to_datetime(positions[j].time, unit='s')
        #         current_time = pd.to_datetime(dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        #         # df = SF.collect_price(pair=symbol, api='Synthetic', days=1, granularity="M1")
        #         # df = df.set_index('date')
                
        #         high_price = get.close[open_time:current_time].max()
        #         low_price = get.close[open_time:current_time].min()
                
        #         if positions[j].symbol == symbol:                   
        #             # Order type is a buy
        #             if positions[j].type == 0:
                        
        #                 price = positions[j].price_open
                        
        #                 stop_loss = round(price - (multiplier_list[i] * (price / (1 + multiplier_list[i]))), 2)
        #                 take_profit = round(price + (multiplier_list[i] * (price / (1 + multiplier_list[i]*1.2))), 2)
                        
        #                 if high_price > take_profit:                       
        #                     # SF.close_single_pair(symbol, positions[j])
        #                     SF.close_all_trades(symbol, positions)
        #                     print(f'Symbol {symbol} closed at Profit')
        #                 elif low_price < stop_loss:  
        #                     SF.close_single_pair(symbol, positions[j])
        #                     # SF.close_all_trades(symbol, positions)
        #                     print(f'Symbol {symbol} closed at Loss')
                            
                            
        #                 # elif df_01['Stoch'].iloc[-1] == 1:
        #                 #     # SF.close_single_pair(symbol, positions[j])
        #                 #     SF.close_all_trades(symbol, positions)
        #                 #     print(f'Symbol {symbol} closed at Edge')
                            
                            
        #             # Order type is a sell    
        #             elif positions[j].type == 1:
                        
        #                 price = positions[j].price_open
                        
        #                 stop_loss = round(price + (multiplier_list[i] * (price / (1 + multiplier_list[i]))), 2)
        #                 take_profit = round(price - (multiplier_list[i] * (price / (1 + multiplier_list[i]*1.2))), 2)
                        
        #                 if low_price < take_profit:                       
        #                     # SF.close_single_pair(symbol, positions[j])
        #                     SF.close_all_trades(symbol, positions)
        #                     print(f'Symbol {symbol} closed at Profit')
        
        #                 elif high_price > stop_loss:  
        #                     SF.close_single_pair(symbol, positions[j])
        #                     # SF.close_all_trades(symbol, positions)
        #                     print(f'Symbol {symbol} closed at Loss')
                            
                        # elif df_01['Stoch'].iloc[-1] == 2:
                        #     # SF.close_single_pair(symbol, positions[j])
                        #     SF.close_all_trades(symbol, positions)
                        #     print(f'Symbol {symbol} closed at Edge')
        # print(get['Final_signal'].iloc[-1])
    time.sleep(60)  # Wait before checking the next trade



KeyboardInterrupt: 

In [ ]:
# Shutdown MT5 connection
mt5.shutdown()

NameError: name 'mt5' is not defined